# 🏁 Formula 1 Workshop: Data Setup

*This notebook prepares all data needed for the Databricks 101 Workshop by downloading F1 datasets and loading them into a volume.*

---

## 🎯 What You'll Do

This setup notebook will:
1. **Create a Volume** to store raw data files
2. **Download F1 datasets** from GitHub 
3. **Save CSV files** to the Volume
4. **Verify data** is properly loaded

**Note:** Run this notebook first before proceeding to the workshop materials.

## 1️⃣ Create a Volume for Raw Data Storage

First, let's create a volume in the Unity Catalog to store our raw F1 data files. 
Volumes are ideal for storing unstructured data like CSV files.

In [ ]:
# Create a volume to store our raw data
# We'll use the main catalog and default schema

try:
    # Check if volume already exists
    spark.sql("DESCRIBE VOLUME main.default.f1_raw_data")
    print("✅ Volume 'main.default.f1_raw_data' already exists.")
    
except Exception as e:
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        # Volume doesn't exist, create it
        spark.sql("CREATE VOLUME IF NOT EXISTS main.default.f1_raw_data")
        print("✅ Volume 'main.default.f1_raw_data' created successfully.")
    else:
        # Some other error occurred
        print(f"❌ Error: {str(e)}")

# Display the volume information
display(spark.sql("DESCRIBE VOLUME main.default.f1_raw_data"))

## 2️⃣ Download F1 Datasets from GitHub

Next, we'll download the Formula 1 datasets from the GitHub repository.
We'll focus on these key files:
- **races.csv** - Information about F1 races
- **drivers.csv** - Information about F1 drivers
- **results.csv** - Race results and performance data

In [ ]:
# Import necessary libraries
import requests
import os
from pyspark.sql import functions as F

# Set the GitHub repository URL
github_repo = "https://github.com/toUpperCase78/formula1-datasets"
raw_content_base = "https://raw.githubusercontent.com/toUpperCase78/formula1-datasets/master"

# Define the files we want to download
files_to_download = [
    "races.csv",
    "drivers.csv", 
    "results.csv"
]

# Set volume path
volume_path = "/Volumes/main/default/f1_raw_data/"

# Ensure the volume directory exists
os.makedirs(volume_path, exist_ok=True)

# Function to download and save files
def download_and_save(filename):
    # Construct the URL
    url = f"{raw_content_base}/{filename}"
    
    try:
        # Download the file
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Save to volume
        file_path = os.path.join(volume_path, filename)
        with open(file_path, "wb") as f:
            f.write(response.content)
            
        # Get file size for verification
        file_size = os.path.getsize(file_path)
        
        return {
            "filename": filename,
            "status": "Success",
            "size_bytes": file_size,
            "path": file_path
        }
        
    except Exception as e:
        return {
            "filename": filename,
            "status": "Failed",
            "error": str(e),
            "path": None
        }

# Download each file and collect results
download_results = []
for file in files_to_download:
    print(f"Downloading {file}...")
    result = download_and_save(file)
    download_results.append(result)
    if result["status"] == "Success":
        print(f"✅ Successfully downloaded {file} ({result['size_bytes']:,} bytes)")
    else:
        print(f"❌ Failed to download {file}: {result['error']}")

# Create a DataFrame to display the results
download_df = spark.createDataFrame(download_results)
display(download_df)

## 3️⃣ Verify File Contents

Let's take a quick look at the files we've downloaded to ensure they contain the expected data.

In [ ]:
# List all files in the volume
import os
volume_files = os.listdir(volume_path)

print(f"Files in {volume_path}:")
for file in volume_files:
    file_path = os.path.join(volume_path, file)
    file_size = os.path.getsize(file_path)
    print(f"- {file} ({file_size:,} bytes)")

# Function to peek at CSV files
def peek_csv(file_path, num_lines=5):
    with open(file_path, 'r') as f:
        lines = [f.readline().strip() for _ in range(num_lines)]
    return lines

# Peek at each CSV file
for file in files_to_download:
    if file in volume_files:
        file_path = os.path.join(volume_path, file)
        print(f"\n📄 {file} (first 5 lines):")
        lines = peek_csv(file_path)
        for i, line in enumerate(lines):
            print(f"  {i+1}: {line[:100]}..." if len(line) > 100 else f"  {i+1}: {line}")

## 4️⃣ Create Schema Directory for DLT

Delta Live Tables requires a schema location directory. Let's create that structure now.

In [ ]:
# Create directories for DLT schema
dlt_schema_dir = os.path.join(volume_path, "dlt_schema")
os.makedirs(dlt_schema_dir, exist_ok=True)

# Create subdirectories for each table
for file in files_to_download:
    table_name = file.split('.')[0]  # Remove file extension
    table_schema_dir = os.path.join(dlt_schema_dir, table_name)
    os.makedirs(table_schema_dir, exist_ok=True)
    
print(f"✅ Created DLT schema directories in {dlt_schema_dir}")

# List the directory structure
def list_dir_structure(path, indent=0):
    print(' ' * indent + '📁 ' + os.path.basename(path))
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path):
            list_dir_structure(item_path, indent + 2)
        else:
            print(' ' * (indent + 2) + '📄 ' + item)

print("\nVolume Directory Structure:")
list_dir_structure(volume_path)

## 5️⃣ Create a Schema Discovery Function for Future Use

Let's create a function that can be used in other notebooks to preview the schema of our raw CSV files.
This will help when creating tables and transformations.

In [ ]:
# Function to preview file schema
def preview_csv_schema(file_name):
    file_path = f"/Volumes/main/default/f1_raw_data/{file_name}"
    
    # Read the CSV file with header and infer schema
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(file_path)
    
    # Print schema information
    print(f"📊 Schema for {file_name}:")
    df.printSchema()
    
    # Show sample data
    print(f"\n📋 Sample data from {file_name}:")
    df.show(5, truncate=False)
    
    return df

# Preview schemas for our files
for file in files_to_download:
    if file in volume_files:
        preview_csv_schema(file)

## ✅ Setup Complete!

Congratulations! You've successfully:
- ✅ Created a Volume for raw data storage
- ✅ Downloaded F1 datasets from GitHub
- ✅ Saved CSV files to the Volume
- ✅ Verified the data is properly loaded
- ✅ Created necessary directory structure for DLT
- ✅ Previewed data schemas for future use

**Next Steps:**
- Continue to [01_Platform_Tour.ipynb](01_Platform_Tour.ipynb) to start exploring the Databricks platform
- Follow through each notebook in the workshop sequence
- Use the data you've just prepared for your F1 analytics journey!

The following data files are now available in the Volume:
```
/Volumes/main/default/f1_raw_data/
├── races.csv
├── drivers.csv
├── results.csv
└── dlt_schema/
    ├── races/
    ├── drivers/
    └── results/
```